## 라이브러리 & Instuction 정의

In [1]:
import os
import re
import random
import json

# negative용
instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user] and, based on this, predicts which news among the candidate news in [Question] the user is most likely to read.

News is provided by title only. 
News is Norwegian news in Norwegian.

There can be multiple lists in [News of interest to the user], each with five news items.
Of the five news in each list, there is one news that users are most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answer should only return the numbers of the news stories that the user is most likely to read.

<example>
[News of interest to the user]
1. På dette bildet skiller Magnus Carlsen seg ut: - Litt tilfeldig / Kong Harald: - Litt uvirkelig å bli 80 / Se lesernes nyttårsbilder / Måtte i oppvaskmøte etter å ha kritisert landslaget / Her koker det over for Tønseth. Så stakk han fra stadion i sinne.
Of the five news above, the news that the user is most interested in : Se lesernes nyttårsbilder
2. Døddrukne ungdommer, trusler, vold og skadeverk / Istanbul: Politiet jakter gjerningsmann som drepte 39 mennesker nyttårsaften / Bil ble totalvrak etter krasj med bergvegg / Bolig totalskadd i brann / Åpenbart beruset mann i trafikkulykke på Byåsen
Of the five news above, the news that the user is most interested in : Åpenbart beruset mann i trafikkulykke på Byåsen
...

[Questions]
Returns the most likely clickable news among the candidate news for each question based on the user's news interests.
Question 1) 1: Fotballsupporternes opptrinn i Paris ble fordømt av statsministeren. Nå har de fått straffen sin. / 2: Disse vil bli lensmann i Meråker og Frosta / 3: Mann kritisk skadd i MC-ulykke i Akershus / 4: Vurderer å flytte fra Midtbyen / 5: Saktegående trafikk i Moholtlia
Question 2) 1: Får bygge på hytte nær vernet vassdrag / 2: Nidarosdomen har hovedrollen i norsk variant av «Da Vinci-koden» / 3: Høyre vil etablere norsk «kulturkanon» / 4: Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut / 5: Ungdom får tilbake førerkort for å bli yrkessjåfør
...
Don't explain why in your answer, just return the number of the news.

<answer>
Question 1 : 3
Question 2 : 5
...
"""

# positive용
instruction_positive = """
You are a bot that identifies users' news interests from [click history] and, based on this, predicts which news among the candidate news in [Question] the user is most likely to read.

News is provided by title only.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
Answers should only return the number of the ranked news.

<example>
[Click History]
1. click : Se lesernes nyttårsbilder
2. click : Åpenbart beruset mann i trafikkulykke på Byåsen
...

[Questions]
Returns the most likely clickable news among the candidate news for each question based on the user's news interests.
Question 1) 1: Fotballsupporternes opptrinn i Paris ble fordømt av statsministeren. Nå har de fått straffen sin. / 2: Disse vil bli lensmann i Meråker og Frosta / 3: Mann kritisk skadd i MC-ulykke i Akershus / 4: Vurderer å flytte fra Midtbyen / 5: Saktegående trafikk i Moholtlia
Question 2) 1: Får bygge på hytte nær vernet vassdrag / 2: Nidarosdomen har hovedrollen i norsk variant av «Da Vinci-koden» / 3: Høyre vil etablere norsk «kulturkanon» / 4: Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut / 5: Ungdom får tilbake førerkort for å bli yrkessjåfør
...
Don't explain why in your answer, just return the number of the news.

<answer>
Question 1 : 3
Question 2 : 5
...
"""


## 함수 정의

In [2]:
def create_json(purpose, u_numbers, file_name):
    """
    JSONL 파일 생성 함수
    """
    
    # U*.txt 파일과 메타데이터가 포함된 디렉토리 경로
    data_dir = f'user_prompts/{purpose}'
    metadata_dir = os.path.join(data_dir, 'metadata')
    positions_file = os.path.join(metadata_dir, 'hidden_positions.txt')

    if purpose == "only_positive":
        instruction = instruction_positive
    elif purpose == "with_negative":
        instruction = instruction_negative

    # 1단계: hidden_positions.txt 읽기
    hidden_positions = {}
    with open(positions_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # 'U1   : [1, 3, 3, 4]'와 같은 형식의 라인 매칭
            match = re.match(r'(U\d+)\s*:\s*\[([^\]]+)\]', line)
            if match:
                user_id = match.group(1)
                positions_str = match.group(2)
                # 위치를 정수의 리스트로 변환
                positions = [int(pos.strip()) for pos in positions_str.split(',')]
                hidden_positions[user_id] = positions

    # 2단계: 지정된 U 번호의 각 U*.txt 파일 처리
    data = []
    for u_num in u_numbers:
        user_id = f'U{u_num}'
        user_file = f'{user_id}.txt'
        user_file_path = os.path.join(data_dir, user_file)
        if not os.path.exists(user_file_path):
            continue  # 파일이 없으면 스킵
        # 사용자 파일의 내용 읽기
        with open(user_file_path, 'r', encoding='utf-8') as f:
            prompt = f.read()

        # 콘텐츠에서 질문 추출
        # '[Questions]' 섹션 아래의 질문을 가정
        questions_section = re.search(r'\[Questions\](.*)', prompt, re.DOTALL)
        if not questions_section:
            continue  # 질문이 없으면 스킵
        questions_text = questions_section.group(1)
        # 개별 질문으로 분할
        questions = {}
        for match in re.finditer(r'Question\s*(\d+)\)(.*?)(?=Question\s*\d+\)|$)', questions_text, re.DOTALL):
            q_num = int(match.group(1))
            q_content = match.group(2).strip()
            questions[q_num] = q_content

        # 해당 사용자의 정답 위치 가져오기
        positions = hidden_positions.get(user_id, [])
        assistant_content_lines = []
        for idx, (q_num, q_content) in enumerate(sorted(questions.items())):
            # 정답 위치 가져오기
            if idx < len(positions):
                correct_position = positions[idx]
            else:
                # 질문 수보다 위치가 적으면 랜덤으로 정답 위치 할당
                correct_position = random.randint(1, 5)
            # 어시스턴트의 콘텐츠 형식화 (정답 값만 포함)
            assistant_content_lines.append(f"Question {q_num}: {correct_position}")

        assistant_content = '\n'.join(assistant_content_lines)

        # 데이터 항목 구성
        data_entry = {
            "messages": [
                {"role": "system", "content": instruction},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": assistant_content}
            ]
        }
        data.append(data_entry)

    # 이제 'data'에는 지정된 U 번호의 사용자의 처리된 데이터가 포함되어 있음
    # JSONL 형식으로 저장
    output_file = f'gpt_finetuning_data/{file_name}'
    with open(output_file, 'w', encoding='utf-8') as f:
        for data_entry in data:
            json_line = json.dumps(data_entry, ensure_ascii=False)
            f.write(json_line + '\n')

    print(f"처리된 데이터가 {output_file}으로 저장되었습니다.")


## 실행

In [3]:
# 처리할 User 번호를 임의로 지정 (예: U3, U7, U9)
# train_numbers = [i for i in range(1001,1041)]
train_numbers = [1327, 1448, 1280, 1005, 1019, 1044, 1425, 1496, 1295, 1194, 1208, 1074, 1273, 1351, 1083, 1478, 1070, 1267, 1150, 1061, 1105, 1328, 1147, 1374, 1477, 1038, 1433, 1096, 1334, 1275, 1227, 1402, 1463, 1087, 1365, 1494, 1180, 1102, 1487, 1313]
val_numbers = [1055, 1047, 1407, 1484, 1369, 1326, 1126, 1400, 1166, 1480]

create_json('only_positive', train_numbers, "train_positive.jsonl")
create_json('only_positive', val_numbers, "val_positive.jsonl")

create_json('with_negative', train_numbers, "train_negative.jsonl")
create_json('with_negative', val_numbers, "val_negative.jsonl")

처리된 데이터가 gpt_finetuning_data/train_positive.jsonl으로 저장되었습니다.
처리된 데이터가 gpt_finetuning_data/val_positive.jsonl으로 저장되었습니다.
처리된 데이터가 gpt_finetuning_data/train_negative.jsonl으로 저장되었습니다.
처리된 데이터가 gpt_finetuning_data/val_negative.jsonl으로 저장되었습니다.
